In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *

In [ ]:
session = get_active_session()

In [ ]:

DROP DATABASE IF EXISTS snowflake_llm_poc;
CREATE Database snowflake_llm_poc;
use snowflake_llm_poc;

In [ ]:
create or replace stage snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt url="s3://{}/{}/" 
credentials=(aws_key_id=''
aws_secret_key='')
Directory=(ENABLE=TRUE);

In [ ]:
ls @snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt;

In [ ]:
alter stage snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt refresh;

In [ ]:
SELECT * FROM directory(@snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt);

In [ ]:
--read pdf files & text chunking
CREATE OR REPLACE function read_pdf_and_split(file_name string)
  RETURNS ARRAY
  LANGUAGE PYTHON
  RUNTIME_VERSION = '3.12'
  PACKAGES = ('snowflake-snowpark-python','PyPDF2','langchain')
  HANDLER = 'main_fn'
AS
$$
from snowflake.snowpark.files import SnowflakeFile
import PyPDF2
import re
from langchain.text_splitter import CharacterTextSplitter

def main_fn(file_name):
    f = SnowflakeFile.open(file_name, 'rb')
    pdf_object = PyPDF2.PdfReader(f)
    
    # Initialize a variable to hold all the text
    all_text = ""
    
    # Iterate over all the pages and concatenate the text
    for page in pdf_object.pages:
        all_text += page.extract_text().replace('\n',' ')
    
    # split documents into chunks
    text_splitter = CharacterTextSplitter(
    separator='.',
    chunk_size=200,
    chunk_overlap=25,
    
    )
    split_docs = text_splitter.split_text(all_text)
    return split_docs
$$;


In [ ]:
create or replace TABLE snowflake_llm_poc.PUBLIC.DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    SIZE NUMBER(38,0), -- Size of the PDF
    Index Number(38,0), --Index no. of the chunk
    CHUNK VARCHAR(16777216), -- Piece of text
    Embedding_Vector VECTOR(FLOAT, 768)
);

Enable Cross-region interface

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';

In [ ]:
insert into snowflake_llm_poc.PUBLIC.docs_chunks_table (relative_path, size, Index,chunk,Embedding_Vector)
with splitted_data as (SELECT RELATIVE_PATH,SIZE,read_pdf_and_split(BUILD_SCOPED_FILE_URL( @snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt , RELATIVE_PATH )) as pdf_text_split FROM directory(@snowflake_llm_poc.PUBLIC.Snow_stage_directory_table_yt))
select Relative_path,SIZE,f.Index,trim(f.value,'"') as chunk,SNOWFLAKE.CORTEX.EMBED_TEXT_768('e5-base-v2', trim(f.value,'"')) as Embedding_Vector from splitted_data , lateral flatten(pdf_text_split) f ;

In [ ]:

select * from snowflake_llm_poc.PUBLIC.docs_chunks_table;

In [ ]:
SET user_question = 'Safety Precautions for the Mondracer Infant Bike?';
select $user_question;

In [ ]:
SELECT snowflake.cortex.complete(
    'mistral-large', 
    CONCAT( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            select listagg(chunk,', ') from (SELECT CHUNK from snowflake_llm_poc.PUBLIC.docs_chunks_table
            ORDER BY VECTOR_L2_DISTANCE(
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('e5-base-v2', 
            $user_question
            ), Embedding_Vector
            ) limit 5)
        ),
        ' Question: ', 
        $user_question,
        'Answer: '
    )
) as response;

In [ ]:
SET user_question = 'Safety Precautions for the Downhill Bike?';
select $user_question;

In [ ]:
SELECT snowflake.cortex.complete(
    'mistral-large', 
    CONCAT( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            select listagg(chunk,', ') from (SELECT CHUNK from snowflake_llm_poc.PUBLIC.docs_chunks_table
            ORDER BY VECTOR_L2_DISTANCE(
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('e5-base-v2', 
            $user_question
            ), Embedding_Vector
            ) limit 5)
        ),
        ' Question: ', 
        $user_question,
        'Answer: '
    )
) as response;

In [ ]:
SET user_question = 'What is the Safety Precautions for the Downhill Bike &  Infant Bike? How Downhill Bike Rincon del Cielo bike is tested? What is the recommended maximum speed for riding the infant bike & what is the name of the Downhill Bike?';
select $user_question;

In [ ]:
SELECT snowflake.cortex.complete(
    'mistral-large', 
    CONCAT( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            select listagg(chunk,', ') from (SELECT CHUNK from snowflake_llm_poc.PUBLIC.docs_chunks_table
            ORDER BY VECTOR_L2_DISTANCE(
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('e5-base-v2', 
            $user_question
            ), Embedding_Vector
            ) limit 5)
        ),
        ' Question: ', 
        $user_question,
        'Answer: '
    )
) as response;

In [ ]:
def decompose_query(session, user_question):
    """
    Decomposes a complex user question into atomic sub-questions
    using Snowflake Cortex with few-shot prompting.

    Returns:
        List[str]: sub-questions, each ending with a full stop.
    """

    sql = f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'openai-gpt-4.1',
        CONCAT(
            'You are a helpful assistant for a Retrieval Augmented Generation (RAG) system. ',
            'Your task is to decompose a complex user question into simpler, atomic sub-questions. ',

            'Rules: ',
            '- Each sub-question must target exactly one fact or concept. ',
            '- Each sub-question must be independently answerable from documents. ',
            '- Do NOT answer the questions. ',
            '- Do NOT add explanations or headings. ',
            '- Return ONLY the sub-questions. ',
            '- Return one sub-question per line. ',
            '- Each sub-question MUST end with aquestion mark(?). ',

            'Example 1 Query: Did Microsoft or Google make more money last year? ',
            'Output: ',
            'How much profit did Microsoft make last year. ',
            'How much profit did Google make last year. ',

            'Example 2 Query: Where is the Octank company waterfront building located and how does the whistleblower scandal hurt the company and its image? ',
            'Output: ',
            'Where is the Octank waterfront building located. ',
            'What is the whistleblower scandal involving Octank. ',
            'How did the whistleblower scandal affect Octank’s reputation and public image. ',

            'Now decompose the following question. ',
            'Query: ', $$ {user_question} $$
        )
    ) AS decomposed_queries
    """

    result = session.sql(sql).collect()[0]["DECOMPOSED_QUERIES"]

    # Deterministic parsing (guaranteed by the prompt)
    sub_queries = [
        line.strip()
        for line in result.split("\n") if line !=""
    ]

    return sub_queries


In [ ]:
def retrieve_context(session, sub_question, top_k=5):
    sql = f"""
    WITH ranked_chunks AS (
        SELECT
            CHUNK,
            VECTOR_L2_DISTANCE(
                SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                    'e5-base-v2',
                    $$ {sub_question} $$
                ),
                Embedding_Vector
            ) AS distance
        FROM snowflake_llm_poc.PUBLIC.docs_chunks_table
        ORDER BY distance
        LIMIT {top_k}
    )
    SELECT LISTAGG(CHUNK, ' ') AS context
    FROM ranked_chunks
    """

    return session.sql(sql).collect()[0]["CONTEXT"]


In [ ]:
def generate_sub_answer(session, sub_question, context):
    prompt = f"""
Answer the question strictly based on the context below.
If the answer is not present, say "Not found in the document".

Context:
{context}

Question:
{sub_question}

Answer:
"""

    sql = f"""
    SELECT snowflake.cortex.complete(
        'openai-gpt-4.1',
        $$ {prompt} $$
    ) AS answer
    """

    return session.sql(sql).collect()[0]["ANSWER"]

In [ ]:
def synthesize_final_answer(session, original_question, qa_pairs):
    formatted_qa = "\n".join(
        [f"Q: {q}\nA: {a}" for q, a in qa_pairs]
    )

    synthesis_prompt = f"""
You are an expert assistant.

Using the following intermediate question–answer pairs,
synthesize a clear & detailed answer
to the original question.

Original Question:
{original_question}

Intermediate Answers:
{formatted_qa}

Final Answer:
"""

    sql = f"""
    SELECT snowflake.cortex.complete(
        'mistral-large',
        $$ {synthesis_prompt} $$
    ) AS final_answer
    """

    return session.sql(sql).collect()[0]["FINAL_ANSWER"]


In [ ]:
def rag_with_query_decomposition(session, user_question):
    # 1. Decompose
    sub_questions = decompose_query(session, user_question)
    

    qa_pairs = []

    # 2–3. RAG per sub-question
    for sq in sub_questions:
        context = retrieve_context(session, sq)
        answer = generate_sub_answer(session, sq, context)
        qa_pairs.append((sq, answer))

    # 4. Synthesize final answer
    final_answer = synthesize_final_answer(
        session,
        user_question,
        qa_pairs
    )

    return {
        "original_question": user_question,
        "sub_questions": sub_questions,
        "intermediate_answers": qa_pairs,
        "final_answer": final_answer
    }

In [ ]:
user_question = 'What is the Safety Precautions for the Downhill Bike &  Infant Bike? How Downhill Bike Rincon del Cielo bike is tested? What is the recommended maximum speed for riding the infant bike?';
rag_with_query_decomposition(session, user_question)